In [2]:
import pandas as pd
import json 

/var/folders/d5/83lsmmvj4cxcmfbyw6wgy52m0000gn/T/ipykernel_28857/2085782588.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
with open("../input_data/scanned_sboms/Corvus.AzureFunctionsKeepAlive.sbom-20230731145443527.json.json") as file_to_open:
    data = file_to_open.read()
json_data = json.loads(data)

In [4]:
print(json_data.keys())

dict_keys(['SchemaVersion', 'CreatedAt', 'ArtifactName', 'ArtifactType', 'Metadata', 'Results'])


In [5]:
json_data['Results']

[{'Target': '',
  'Class': 'lang-pkgs',
  'Type': 'nuget',
  'Vulnerabilities': [{'VulnerabilityID': 'CVE-2022-26907',
    'PkgID': 'Microsoft.Rest.ClientRuntime@2.3.23',
    'PkgName': 'Microsoft.Rest.ClientRuntime',
    'PkgIdentifier': {'PURL': 'pkg:nuget/Microsoft.Rest.ClientRuntime@2.3.23'},
    'InstalledVersion': '2.3.23',
    'FixedVersion': '2.3.24',
    'Status': 'fixed',
    'Layer': {},
    'SeveritySource': 'ghsa',
    'PrimaryURL': 'https://avd.aquasec.com/nvd/cve-2022-26907',
    'DataSource': {'ID': 'ghsa',
     'Name': 'GitHub Security Advisory NuGet',
     'URL': 'https://github.com/advisories?query=type%3Areviewed+ecosystem%3Anuget'},
    'Title': 'Azure SDK for .NET Information Disclosure Vulnerability.',
    'Description': 'Azure SDK for .NET Information Disclosure Vulnerability',
    'Severity': 'MEDIUM',
    'CweIDs': ['CWE-532'],
    'VendorSeverity': {'ghsa': 2, 'nvd': 2},
    'CVSS': {'ghsa': {'V3Vector': 'CVSS:3.1/AV:N/AC:H/PR:L/UI:N/S:U/C:H/I:N/A:N',
      '

In [7]:
package_vulnerability_raw_data = json_data.get("Results")[0]
vulnerability_to_df = pd.DataFrame(package_vulnerability_raw_data)
vulnerability_df_filtered = vulnerability_to_df.map(lambda x: x[0] if isinstance(x, list) else x)

In [8]:
vulnerability_to_df

,Target,Class,Type,Vulnerabilities
0,,lang-pkgs,nuget,"{'VulnerabilityID': 'CVE-2022-26907', 'PkgID':..."
1,,lang-pkgs,nuget,"{'VulnerabilityID': 'CVE-2024-21907', 'PkgID':..."
2,,lang-pkgs,nuget,"{'VulnerabilityID': 'CVE-2021-26701', 'PkgID':..."
3,,lang-pkgs,nuget,"{'VulnerabilityID': 'CVE-2019-0820', 'PkgID': ..."


In [9]:
df_dict = pd.json_normalize(vulnerability_to_df['Vulnerabilities'])

In [10]:
df_dict

,VulnerabilityID,PkgID,PkgName,InstalledVersion,FixedVersion,Status,SeveritySource,PrimaryURL,Title,Description,...,CVSS.nvd.V3Vector,CVSS.nvd.V2Score,CVSS.nvd.V3Score,VendorSeverity.bitnami,VendorSeverity.oracle-oval,VendorSeverity.redhat,CVSS.bitnami.V3Vector,CVSS.bitnami.V3Score,CVSS.redhat.V3Vector,CVSS.redhat.V3Score
0,CVE-2022-26907,Microsoft.Rest.ClientRuntime@2.3.23,Microsoft.Rest.ClientRuntime,2.3.23,2.3.24,fixed,ghsa,https://avd.aquasec.com/nvd/cve-2022-26907,Azure SDK for .NET Information Disclosure Vuln...,Azure SDK for .NET Information Disclosure Vuln...,...,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:N/A:N,4.0,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CVE-2024-21907,Newtonsoft.Json@11.0.2,Newtonsoft.Json,11.0.2,13.0.1,fixed,ghsa,https://avd.aquasec.com/nvd/cve-2024-21907,Improper Handling of Exceptional Conditions in...,Newtonsoft.Json before version 13.0.1 is affec...,...,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CVE-2021-26701,System.Text.Encodings.Web@4.5.0,System.Text.Encodings.Web,4.5.0,"4.5.1, 4.7.2, 5.0.1",fixed,ghsa,https://avd.aquasec.com/nvd/cve-2021-26701,dotnet: System.Text.Encodings.Web Remote Code ...,.NET Core Remote Code Execution Vulnerability,...,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,7.5,9.8,3.0,3.0,3.0,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H,8.1,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H,8.1
3,CVE-2019-0820,System.Text.RegularExpressions@4.3.0,System.Text.RegularExpressions,4.3.0,4.3.1,fixed,ghsa,https://avd.aquasec.com/nvd/cve-2019-0820,dotnet: timeouts for regular expressions are n...,A denial of service vulnerability exists when ...,...,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H,5.0,7.5,NaN,3.0,2.0,NaN,NaN,CVSS:3.0/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H,7.5


In [11]:
df_final = vulnerability_to_df.drop('Vulnerabilities', axis=1).join(df_dict)

In [13]:
df_final['sbom_name'] = json_data.get('ArtifactName').split('/')[-1]

In [14]:
df_final

,Target,Class,Type,VulnerabilityID,PkgID,PkgName,InstalledVersion,FixedVersion,Status,SeveritySource,...,CVSS.nvd.V2Score,CVSS.nvd.V3Score,VendorSeverity.bitnami,VendorSeverity.oracle-oval,VendorSeverity.redhat,CVSS.bitnami.V3Vector,CVSS.bitnami.V3Score,CVSS.redhat.V3Vector,CVSS.redhat.V3Score,sbom_name
0,,lang-pkgs,nuget,CVE-2022-26907,Microsoft.Rest.ClientRuntime@2.3.23,Microsoft.Rest.ClientRuntime,2.3.23,2.3.24,fixed,ghsa,...,4.0,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...
1,,lang-pkgs,nuget,CVE-2024-21907,Newtonsoft.Json@11.0.2,Newtonsoft.Json,11.0.2,13.0.1,fixed,ghsa,...,NaN,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...
2,,lang-pkgs,nuget,CVE-2021-26701,System.Text.Encodings.Web@4.5.0,System.Text.Encodings.Web,4.5.0,"4.5.1, 4.7.2, 5.0.1",fixed,ghsa,...,7.5,9.8,3.0,3.0,3.0,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H,8.1,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H,8.1,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...
3,,lang-pkgs,nuget,CVE-2019-0820,System.Text.RegularExpressions@4.3.0,System.Text.RegularExpressions,4.3.0,4.3.1,fixed,ghsa,...,5.0,7.5,NaN,3.0,2.0,NaN,NaN,CVSS:3.0/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H,7.5,Corvus.AzureFunctionsKeepAlive.sbom-2023073114...


In [15]:
df_final.to_csv('../results/vulnerability_report.csv')


## Get Packages Data

CSV Schema for vulnerability data:
|id |title|description|cwe|cve|severity|epss.cve|epss.epss|epss.percentile|epss.date|
|---|-----|-----------|---|---|--------|--------|---------|---------------|---------|
|   |     |           |   |   |        |        |         |               |         |


## Import component location data

In [124]:
join_table.to_csv('../results/vulnerability_report.csv')
join_table.to_json('../results/vulnerability_report.json')

## Get Summary Data

In [9]:
summary_raw_data = json_data.get("summary")
summary_to_df = pd.DataFrame(summary_raw_data, index=[0])

In [10]:
summary_to_df

,Unspecified,Low,Moderate,High,Critical
0,0,0,2,12,2


In [ ]:
summary_to_df.to_csv('../results/summary_report.csv')
summary_to_df.to_json('../results/summary_report.json')